### Libraries

In [3]:
import numpy
import numpy as np
import random
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from keras.layers import Activation
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Lambda, Layer, ReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LSTM, SpatialDropout2D, Concatenate
tf.keras.layers.Concatenate()
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, UpSampling2D, BatchNormalization
from keras.layers.core import Reshape
from keras import optimizers
from tensorflow.keras import regularizers
from keras import Input, Model
from time import time
import time as tm
from keras.initializers import Constant, RandomNormal, glorot_normal
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import l2
from keras import backend as K
from tensorflow.keras.utils import plot_model
from keras.layers import  concatenate

from tfkan.layers import DenseKAN
from tfkan.layers import Conv2DKAN
from tfkan.layers import base

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except RuntimeError as e:
        print(e)


In [5]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from keras.layers import Activation
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Lambda, Layer, ReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LSTM, SpatialDropout2D, Concatenate
tf.keras.layers.Concatenate()
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, UpSampling2D, BatchNormalization
from keras.layers.core import Reshape
from keras import optimizers
from tensorflow.keras import regularizers
from keras import Input, Model
from time import time
import time as tm
from keras.initializers import Constant, RandomNormal, glorot_normal
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import l2
from keras import backend as K
from tensorflow.keras.utils import plot_model
from keras.layers import  concatenate

from tfkan.layers import DenseKAN
from tfkan.layers import Conv2DKAN
from tfkan.layers import base

### 30 SRM filters for preprocessing and the activation function

In [7]:
################################################## 30 SRM FILTERS
srm_weights = np.load('SRM_Kernels.npy') 
biasSRM=numpy.ones(30)
print (srm_weights.shape)
################################################## TLU ACTIVATION FUNCTION
T3 = 3;
def Tanh3(x):
    tanh3 = K.tanh(x)*T3
    return tanh3

(5, 5, 1, 30)


### Functions

In [8]:
# Squeeze-and Excitation

def squeeze_excitation_layer(input_layer, out_dim, ratio, conv):
  squeeze = tf.keras.layers.GlobalAveragePooling2D()(input_layer)
  excitation = DenseKAN(units=int(out_dim / ratio))(squeeze)

  reshape_dim = tf.keras.backend.int_shape(excitation)[-1]

  excitation = tf.keras.layers.Reshape([1,1,reshape_dim])(excitation)
  excitation = Conv2DKAN(out_dim, 1, strides=1, padding="same")(excitation)

  scale = tf.keras.layers.multiply([input_layer, excitation])

  if conv:
    shortcut = tf.keras.layers.Conv2DKAN(out_dim,kernel_size=1,strides=1,
                                      padding='same')(input_layer)
    shortcut = tf.keras.layers.BatchNormalization()(shortcut)
  else:
    shortcut = input_layer
  out = tf.keras.layers.add([shortcut, scale])
  return out




def sConvsame(input,parameters,size,nstrides):
  return Conv2DKAN(parameters, size, strides=nstrides, padding="same")(input)

def sConv(input,parameters,size,nstrides):
  return Conv2DKAN(parameters, size, strides=nstrides)(input)

def sBN (input):
  return tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=True, trainable=True)(input)

def sGlobal_Avg_Pooling (input):
  return tf.keras.layers.GlobalAveragePooling2D()(input)

def sDense (input, n_units):
  return DenseKAN(n_units)(input)

def smultiply (input_1, input_2):
  return tf.keras.layers.multiply([input_1, input_2])

def sadd (input_1, input_2):
  return tf.keras.layers.add([input_1, input_2])



def Block_1 (input, parameter):
  output = sConv(input, parameter, 3, 1)
  output = sBN(output)
  return output



def SE_Block(input, out_dim, ratio):
  output = sGlobal_Avg_Pooling(input)
  output = sDense(output, int(out_dim/ratio))
  return output



def Block_2 (input, parameter):
  output = sConvsame(input, parameter, 3, 1)
  output = sBN(output)
  multiplier = SE_Block(output,  parameter, parameter)
  # output = smultiply(output, output)
  output = smultiply(multiplier, output)
  output = sadd(output, input)
  return output


from tensorflow.python.ops.gen_array_ops import shape

def Block_3 (input, parameter):
  addition = sConv(input, parameter, 1, 2)
  addition = sBN(addition)
  output = sConvsame(input, parameter, 3, 2)
  output = sBN(output)
  output = sConvsame(output, parameter, 3, 1)
  output = sBN(output)
  multiplier = SE_Block(output,  parameter, parameter)
  output = smultiply(multiplier, output)
  addition = sConv(addition, parameter, 1, 1)
  output = sadd(output, addition)
  return output

def Block_4 (input, parameter):
  output = Block_1(input, parameter)
  output = sConv(input, parameter, 3, 1)
  output = sBN(output)

  return output


In [9]:
# ViT ARCHITECTURE
#Hyperparameters 1 tRANSFORMER
# ViT ARCHITECTURE
LAYER_NORM_EPS_1 = 1e-6
PROJECTION_DIM_1 = 16
NUM_HEADS_1 = 4
NUM_LAYERS_1 = 4
MLP_UNITS_1 = [
    PROJECTION_DIM_1 * 2,
    PROJECTION_DIM_1,
]
# OPTIMIZER
LEARNING_RATE_2 = 1e-3
WEIGHT_DECAY_2 = 1e-4

IMAGE_SIZE_2 =  12# We will resize input images to this size.
PATCH_SIZE_2 = 4  # Size of the patches to be extracted from the input images.
NUM_PATCHES_2 = (IMAGE_SIZE_2 // PATCH_SIZE_2) ** 2
print(NUM_PATCHES_2)
# ViT ARCHITECTURE
LAYER_NORM_EPS_2 = 1e-6
PROJECTION_DIM_2 = 128
NUM_HEADS_2 = 4
NUM_LAYERS_2 = 4
MLP_UNITS_2 = [
    PROJECTION_DIM_2 * 2,
    PROJECTION_DIM_2
]


def position_embedding(projected_patches, num_patches=NUM_PATCHES_2, projection_dim=PROJECTION_DIM_2):
    # Build the positions.

    positions = tf.range(start=0, limit=num_patches, delta=1)

    # Encode the positions with an Embedding layer.
    encoded_positions = layers.Embedding(
        input_dim=num_patches, output_dim=projection_dim
    )(positions)

    # Add encoded positions to the projected patches.
    return projected_patches + encoded_positions

def mlp(x, dropout_rate, hidden_units):
    # Iterate over the hidden units and
    # add Dense => Dropout.
    for units in hidden_units:
        x = DenseKAN(units)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

def transformer_1(encoded_patches):
    # Layer normalization 1.
    x1 = layers.LayerNormalization(epsilon=LAYER_NORM_EPS_1)(encoded_patches)

    # Multi Head Self Attention layer 1.
    attention_output = layers.MultiHeadAttention(
        num_heads=NUM_HEADS_1, key_dim=PROJECTION_DIM_1, dropout=0.1
    )(x1, x1)

    # Skip connection 1.
    x2 = layers.Add()([attention_output, encoded_patches])

    # Layer normalization 2.
    x3 = layers.LayerNormalization(epsilon=LAYER_NORM_EPS_1)(x2)

    # MLP layer 1.
    x4 = mlp(x3, hidden_units=MLP_UNITS_1, dropout_rate=0.1)

    # Skip connection 2.
    encoded_patches = layers.Add()([x4, x2])
    return encoded_patches

def Transform_sh_1(inputs):
    projected_patches = Conv2DKAN(filters=512,kernel_size=9,strides=1,padding="same" )(inputs)
    _, h, w, c = projected_patches.shape
    print(c)
    projected_patches = layers.Reshape((h * w, c))(
        projected_patches
    )  # (B, number_patches, projection_dim)

    # Iterate over the number of layers and stack up blocks of
    # Transformer.
    for i in range(NUM_LAYERS_1):
        # Add a Transformer block.
        encoded_patches = transformer_1(projected_patches)
        # Add TokenLearner layer in the middle of the
        # architecture. The paper suggests that anywhere
        # between 1/2 or 3/4 will work well.
        _, hh, c = encoded_patches.shape
        h = int(math.sqrt(hh))
        encoded_patches = layers.Reshape((h, h, c))(encoded_patches)
    print(encoded_patches.shape)
        #print(encoded_patches.shape)
    return encoded_patches

def transformer_2(encoded_patches):

    x1 = layers.LayerNormalization(epsilon=LAYER_NORM_EPS_2)(encoded_patches)

    # Multi Head Self Attention layer 1.
    attention_output = layers.MultiHeadAttention(
        num_heads=NUM_HEADS_2, key_dim=PROJECTION_DIM_2, dropout=0.1
    )(x1, x1)

    # Skip connection 1.
    x2 = layers.Add()([attention_output, encoded_patches])

    # Layer normalization 2.
    x3 = layers.LayerNormalization(epsilon=LAYER_NORM_EPS_2)(x2)

    # MLP layer 1.
    x4 = mlp(x3, hidden_units=MLP_UNITS_2, dropout_rate=0.1)

    # Skip connection 2.
    encoded_patches = layers.Add()([x4, x2])
    return encoded_patches

def Transform_sh_2(inputs):
    inputs1 = squeeze_excitation_layer(inputs, out_dim=56, ratio=32.0, conv=False)
    print('input',inputs1.shape)
    projected_patches = Conv2DKAN(
          filters=PROJECTION_DIM_2,
          kernel_size=(PATCH_SIZE_2),
          strides=(PATCH_SIZE_2),
          padding="VALID",
      )(inputs1)
    _, h, w, c = projected_patches.shape
    projected_patches = tf.keras.layers.Reshape((h * w, c))(
          projected_patches
      )  # (B, number_patches, projection_dim)
      # Add positional embeddings to the projected patches.
    print('projected',projected_patches.shape)
    encoded_patches = position_embedding(
          projected_patches
      )  # (B, number_patches, projection_dim)

    encoded_patches = tf.keras.layers.Dropout(0.1)(encoded_patches)

      # Iterate over the number of layers and stack up blocks of
      # Transformer.
    for i in range(NUM_LAYERS_2):
          # Add a Transformer block.
        encoded_patches = transformer_2(encoded_patches)

    return encoded_patches

9


### Model

In [30]:
def kan_arch( img_size=256, compile=True):
    
    tf.keras.backend.clear_session()
    print ("using",2,"classes")
    
    #Preprocessing
    inputs = tf.keras.Input(shape=(256,256,1), name="input_1")
    layers_ty = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(5,5), padding='same', trainable=False, activation=Tanh3, use_bias=True)(inputs)
    layers_tn = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(5,5), padding='same', trainable=True, activation=Tanh3, use_bias=True)(inputs)

    layers = tf.keras.layers.add([layers_ty, layers_tn])
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True)(layers)
    
    #Block 1
    #Layer 0
    layers = Conv2DKAN(filters=4, kernel_size=3, strides=3, padding='same')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=True, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    # #Block 2
    # #Layer 1
    layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = Conv2DKAN(filters=8, kernel_size=3, strides=3, padding='same')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=True, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)  
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= 3)(layers)

    # #Block 3
    # #Layer 2
    layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = Conv2DKAN(filters=16, kernel_size=3, strides=2, padding='same')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=True, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)  
    layers = tf.keras.layers.AveragePooling2D((1,1), strides= 3)(layers)


    layers = tf.keras.layers.GlobalAveragePooling2D()(layers)
    
    #Layer 18
    layers = DenseKAN(2)(layers)
    #Layer 19
    predictions = tf.keras.layers.Softmax(axis=1)(layers)
    model =tf.keras.Model(inputs = inputs, outputs=predictions)
    #Compile
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.95)
    
    if compile:
        model.compile(optimizer= optimizer,
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
        print ("kan model")
    return model

In [31]:
model = kan_arch()
model.summary()

using 2 classes
kan model
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 52, 52, 30)   780         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 52, 52, 30)   780         ['input_1[0][0]']                
                                                                                                  
 add (Add)                      (None, 52, 52, 30)   0           ['c

### Train Functions

In [32]:

def train(model, X_train, y_train, X_valid, y_valid, X_test, y_test, batch_size, epochs, initial_epoch = 0, model_name=""):
    start_time = tm.time()
    log_dir="D:/modelos/"+model_name+"_"+"{}".format(time())
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir)
    filepath = log_dir+"/best-model.hdf5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
    model.reset_states()
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath, 
        monitor='val_accuracy', 
        save_best_only=True, 
        mode='max',
        verbose=1
    )
    #  
    history=model.fit(X_train, y_train, epochs=epochs, 
                        callbacks=[tensorboard,  checkpoint],
                        batch_size=batch_size,validation_data=(X_valid, y_valid),initial_epoch=initial_epoch)
    
    metrics = model.evaluate(X_test, y_test, verbose=0)
    results_dir="D:/modelos//"+model_name+"/"
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
      
        plt.figure(figsize=(10, 10))
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Accuracy Vs Epochs')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.grid('on')
        plt.savefig(results_dir+'Accuracy_Xu_Net_'+model_name+'.eps', format='eps')
        plt.savefig(results_dir+'Accuracy_Xu_Net_'+model_name+'.svg', format='svg')
        plt.savefig(results_dir+'Accuracy_Xu_Net_'+model_name+'.pdf', format='pdf')
        plt.show()
        
        plt.figure(figsize=(10, 10))
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Loss Vs Epochs')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.grid('on')
        plt.savefig(results_dir+'Loss_Xu_Net_'+model_name+'.eps', format='eps')
        plt.savefig(results_dir+'Loss_Xu_Net_'+model_name+'.svg', format='svg')
        plt.savefig(results_dir+'Loss_Xu_Net_'+model_name+'.pdf', format='pdf')
        plt.show()

    TIME = tm.time() - start_time
    print("Time "+model_name+" = %s [seconds]" % TIME)
    return {k:v for k,v in zip (model.metrics_names, metrics)}

### Database Uniward

In [33]:
#Train
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
#Valid
X_valid = np.load('X_valid.npy')
y_valid = np.load('y_valid.npy')
#Test
X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')

X_train = np.transpose(X_train, (0, 2, 3, 1)) 
X_valid = np.transpose(X_valid, (0, 2, 3, 1))

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)

(8000, 256, 256, 1)
(8000, 2)
(2000, 256, 256, 1)
(2000, 2)
(10000, 256, 256, 1)
(10000, 2)


### Trainig

In [34]:
base_name="04S-UNIWARD"
model= kan_arch() 
name="Model_"+'KAN'+"_"+base_name

# history = model.fit(X_train, y_train, epochs=2, batch_size=32, validation_data=(X_valid, y_valid))
_, history  = train(model, X_train, y_train, X_valid, y_valid, X_test, y_test, batch_size=32, epochs=100, model_name=name)

using 2 classes
kan model
Epoch 1/100
250/250 [==============================] - ETA: 0s - loss: 0.7110 - accuracy: 0.4876
Epoch 1: val_accuracy improved from -inf to 0.50300, saving model to D:/modelos/Model_KAN_04S-UNIWARD_1728340539.45035\best-model.hdf5
250/250 [==============================] - 19s 61ms/step - loss: 0.7110 - accuracy: 0.4876 - val_loss: 0.6961 - val_accuracy: 0.5030
Epoch 2/100
249/250 [============================>.] - ETA: 0s - loss: 0.6969 - accuracy: 0.4949
Epoch 2: val_accuracy did not improve from 0.50300
250/250 [==============================] - 14s 58ms/step - loss: 0.6969 - accuracy: 0.4952 - val_loss: 0.6942 - val_accuracy: 0.4985
Epoch 3/100
250/250 [==============================] - ETA: 0s - loss: 0.6970 - accuracy: 0.4849
Epoch 3: val_accuracy improved from 0.50300 to 0.51700, saving model to D:/modelos/Model_KAN_04S-UNIWARD_1728340539.45035\best-model.hdf5
250/250 [==============================] - 15s 58ms/step - loss: 0.6970 - accuracy: 0.4849 - 

KeyboardInterrupt: 